In [1]:
import time
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
from dnn_app_utils_v3 import *
import math


In [2]:
def load_data():
    train_dataset = h5py.File('train_catvnoncat.h5')
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('test_catvnoncat.h5')
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [3]:
train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes=load_data()

In [4]:
train_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0], -1).T   # The "-1" makes reshape flatten the remaining dimensions
test_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0], -1).T

# Standardize data to have feature values between 0 and 1.
train_x = train_x_flatten/255
test_x = test_x_flatten/255
print(train_x.shape,train_set_y_orig.shape)

(12288, 209) (1, 209)


In [5]:
def intialize_parameters(n_x,n_y,n_h):
    w1=np.random.randn(n_h,n_x)
    b1=np.zeros((n_h,1))
    w2=np.random.randn(n_y,n_h)
    b2=np.zeros((n_y,1))
    return w1,b1,w2,b2
def relu(Z):
    F=np.maximum(0,Z)
    cache=Z
    return F,Z
def linear_forward(A_prev,W,b,activation):
    Z=W.dot(A_prev)+b
    if activation=="relu":
        
        
        F,Z=relu(Z)
        return F,Z,A_prev,W,b
    else:
        Z = np.clip(Z, -500, 500)
        sig=1/(1+np.exp(-Z))
        return sig,Z,A_prev,W,b
def linear_backward(dA,Z,W,b,activation,A):
    if activation=="relu":
        
        dZ = np.array(dA, copy=True) 
        dZ[Z <= 0] = 0 
    else:
        s = 1/(1+np.exp(-Z))
        dZ = dA * s * (1-s)
    m = dA.shape[1]
    
    dW = 1./m * np.dot(dZ,A.T)
    db = 1./m * np.sum(dZ, axis = 1, keepdims = True)
    dA_prev = np.dot(W.T,dZ)
    
    return dA_prev, dW, db

    

In [6]:
!jt -t monokai -T -N -kl


In [7]:
def L_forward(X,parameters):
    A=X
    
    
    A_cache=[]
    l=len(parameters)
    for i in range(l-1):
        A_prev=A
        A_cache.append(A)
        A,p,q,w1,b1=linear_forward(A_prev,parameters[i][0],parameters[i][1],"relu")
        
    A_cache.append(A)
    AL,p,q,w1,b1=linear_forward(A,parameters[l-1][0],parameters[l-1][1],"sig")
    A_cache.append(A)
    return AL,A_cache

In [8]:
def initialize_parameters_deep(layer_dims):
    
    
    
    parameters = []
    L = len(layer_dims)            # number of layers in the network

    for i in range(0, L-1):
        w1 = np.random.randn(layer_dims[i+1], layer_dims[i]) * np.sqrt(2/layer_dims[i]) #*0.01
        b1 = np.zeros((layer_dims[i+1], 1))
        parameters.append([w1,b1])
        
        
    return parameters


In [9]:
layer_dims = [12288, 20, 7, 5, 1]


In [10]:
def l_backward(Y,A_mat,AL,caches):
    L = len(A_mat)
    
    grads=[]
    dAL = - (np.divide(Y, AL+0.000001) - np.divide(1 - Y, 1 - AL+0.000001))
    da_prev,dw,db=linear_backward(dAL,caches[L-2][0].dot(A_mat[L-2])+caches[L-2][1],caches[L-2][0],caches[L-2][1],"sig",A_mat[L-2])    
    grads.append([dw,db])
    for i in range(1,L-1):
        
        da_prev,dw,db=linear_backward(da_prev,caches[L-2-i][0].dot(A_mat[L-2-i])+caches[L-2-i][1],caches[L-2-i][0],caches[L-2-i][1],"relu",A_mat[L-2-i])
        grads.append([dw,db])
    return grads

In [11]:
def implement_l(X,Y,layer_dims,alpha,itera):
    pr=initialize_parameters_deep(layer_dims)
    for i in range(itera):
        AL,Z_cache=L_forward(X,pr)
        
        
        grads=l_backward(Y,Z_cache,AL,pr)
        for p in range(0,len(pr)):
            
            pr[p][0]=pr[p][0]-alpha*grads[-(p+1)][0]
            pr[p][1]=pr[p][1]-alpha*grads[-(p+1)][1]
    return pr,AL

In [12]:
def implement_l_without_initial(X,Y,layer_dims,alpha,itera,pr):
    for i in range(itera):
        AL,Z_cache=L_forward(X,pr)
        
        
        grads=l_backward(Y,Z_cache,AL,pr)
        for p in range(0,len(pr)):
            
            pr[p][0]=pr[p][0]-alpha*grads[-(p+1)][0]
            pr[p][1]=pr[p][1]-alpha*grads[-(p+1)][1]
    return pr,AL
    

In [34]:
pr,AL=implement_l(train_x,train_set_y_orig,layer_dims,0.01,800)
q = np.zeros((1,209))

for i in range(AL.shape[1]):
    q[0,i]=(AL[:,i]>0.5)

np.sum(q==train_set_y_orig)/209

C:\Users\shash\AppData\Local\Temp\ipykernel_10164\2225162652.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  q[0,i]=(AL[:,i]>0.5)


0.7129186602870813

In [14]:
def mini_batch(X,Y,batch_size):
    
    k=list(np.random.permutation(int(X.shape[1])))
    shuff_x=X[:,k]
    shuff_y=Y[:,k]
    p=math.ceil(X.shape[1]/batch_size)
    mini_batch_x=[]
    mini_batch_y=[]
    for i in range(p):
        mini_batch_x.append(shuff_x[:,i*batch_size:i*batch_size+batch_size])
        mini_batch_y.append(shuff_y[:,i*batch_size:i*batch_size+batch_size])
    
    batch = (mini_batch_x, mini_batch_y)
    return batch

In [15]:
batch=mini_batch(train_x,train_set_y_orig,8)

In [30]:
pr=initialize_parameters_deep(layer_dims)
AL_final=[]

batch=mini_batch(train_x,train_set_y_orig,8)
o=[]
for k in batch[1]:
    o.append(k)
shuffl_y=np.hstack(o)[0]
AL_final=[]
for i in range(len(batch[0])):
     pr,AL=implement_l_without_initial(batch[0][i],batch[1][i],layer_dims,0.01,100,pr)
     AL_final=AL_final+list(AL)
final_a=(np.hstack(AL_final))
print(final_a,shuffl_y)


[4.89323082e-03 6.66706414e-03 9.92127307e-01 9.95521848e-01
 1.43361013e-02 1.85209879e-02 9.91545760e-01 9.74679580e-01
 9.93150523e-01 9.81806476e-01 9.90498367e-01 4.26946842e-03
 9.98101300e-01 1.53553646e-02 2.76545686e-02 1.57079634e-02
 1.95029036e-02 5.32679897e-01 9.08206672e-03 6.06121540e-03
 5.46616453e-03 1.71985570e-04 8.73768217e-04 5.32679897e-01
 3.94969763e-03 5.74405456e-01 5.74405456e-01 9.32795290e-03
 1.98726163e-02 5.74405456e-01 2.52401084e-02 1.16986607e-02
 1.58181331e-03 3.31943582e-03 3.17528683e-03 4.14007527e-03
 8.93730491e-03 9.84213772e-01 5.90389187e-01 1.04351683e-04
 1.58187521e-03 2.75175383e-02 1.70658268e-03 9.71885866e-04
 8.59911077e-03 6.19716905e-01 6.19716905e-01 1.68775047e-03
 6.59842710e-01 6.59842710e-01 6.59842710e-01 6.59842710e-01
 6.59842710e-01 6.59842710e-01 6.59842710e-01 6.59842710e-01
 2.64336401e-02 9.32208474e-01 3.31213661e-02 6.17974534e-01
 6.17974534e-01 2.76496846e-02 3.18702284e-02 3.62438046e-02
 6.53257123e-01 4.803982

array([[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
        0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
        1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
        1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0,
        0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1,
        0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [31]:
q = np.zeros((1,209))
pred=[]
for i in range(209):
    if final_a[i]>0.5:
        pred.append(1)
    else:
        pred.append(0)



In [32]:

p=0
for i in range(209):
    if shuffl_y[i]==pred[i]:
        p=p+1

In [33]:
p

186